In [3]:
import tensorflow as tf

# 定义标量和张量

In [4]:
# 定义一个随机数（标量）
random_float = tf.random.uniform(shape=())

# 定义一个有2个元素的零向量
zero_vector = tf.zeros(shape=(2))

# 定义两个2×2的常量矩阵
A = tf.constant([[1., 2.], [3., 4.]])
B = tf.constant([[5., 6.], [7., 8.]])

In [5]:
# 查看矩阵A的形状、类型和值
print(A.shape)      # 输出(2, 2)，即矩阵的长和宽均为2
print(A.dtype)      # 输出<dtype: 'float32'>
print(A.numpy())    # 输出[[1. 2.]
                    #      [3. 4.]]
    
C = tf.add(A, B)    # 计算矩阵A和B的和
D = tf.matmul(A, B) # 计算矩阵A和B的乘积

print(C.numpy())
print(D.numpy())

(2, 2)
<dtype: 'float32'>
[[1. 2.]
 [3. 4.]]
[[ 6.  8.]
 [10. 12.]]
[[19. 22.]
 [43. 50.]]


# 自动求导机制

In [6]:
x = tf.Variable(initial_value=3.)
with tf.GradientTape() as tape:     # 在 tf.GradientTape() 的上下文内，所有计算步骤都会被记录以用于求导
    y = tf.square(x)
y_grad = tape.gradient(y, x)        # 计算y关于x的导数
print(y, '\n', y_grad)

tf.Tensor(9.0, shape=(), dtype=float32) 
 tf.Tensor(6.0, shape=(), dtype=float32)


In [7]:
# 多元求导
X = tf.constant([[1., 2.], [3., 4.]])
y = tf.constant([[1.], [2.]])
w = tf.Variable(initial_value=[[1.], [2.]])
b = tf.Variable(initial_value=1.)
with tf.GradientTape() as tape:
    L = tf.reduce_sum(tf.square(tf.matmul(X, w) + b - y))
w_grad, b_grad = tape.gradient(L, [w, b])        # 计算L(w, b)关于w, b的偏导数
print(L, w_grad, b_grad)

tf.Tensor(125.0, shape=(), dtype=float32) tf.Tensor(
[[ 70.]
 [100.]], shape=(2, 1), dtype=float32) tf.Tensor(30.0, shape=(), dtype=float32)


# 线性回归

年份

2013 2014 2015 2016 2017

房价

12000 14000 15000 16500 17500

In [8]:
import numpy as np

X_raw = np.array([2013, 2014, 2015, 2016, 2017], dtype=np.float32)
y_raw = np.array([12000, 14000, 15000, 16500, 17500], dtype=np.float32)

X = (X_raw - X_raw.min()) / (X_raw.max() - X_raw.min())
y = (y_raw - y_raw.min()) / (y_raw.max() - y_raw.min())

In [9]:
a, b = 0, 0

num_epoch = 10000
learning_rate = 5e-4
for e in range(num_epoch):
    # 手动计算损失函数关于自变量（模型参数）的梯度
    y_pred = a * X + b
    grad_a, grad_b = 2 * (y_pred - y).dot(X), 2 * (y_pred - y).sum()

    # 更新参数
    a, b = a - learning_rate * grad_a, b - learning_rate * grad_b

print(a, b)

0.9763702027872221 0.057564988311377796


In [10]:
X = tf.constant(X)
y = tf.constant(y)

a = tf.Variable(initial_value=0.)
b = tf.Variable(initial_value=0.)
variables = [a, b]

num_epoch = 10000
optimizer = tf.keras.optimizers.SGD(learning_rate=5e-4)
for e in range(num_epoch):
    # 使用tf.GradientTape()记录损失函数的梯度信息
    with tf.GradientTape() as tape:
        y_pred = a * X + b
        loss = tf.reduce_sum(tf.square(y_pred - y))
    # TensorFlow自动计算损失函数关于自变量（模型参数）的梯度
    grads = tape.gradient(loss, variables)
    # TensorFlow自动根据梯度更新参数
    optimizer.apply_gradients(grads_and_vars=zip(grads, variables))

In [11]:
variables

[<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.97637>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.057565063>]

# 建模与训练

在 TensorFlow 中，推荐使用 Keras（ tf.keras ）构建模型。Keras 是一个广为流行的高级神经网络 API，简单、快速而不失灵活性，现已得到 TensorFlow 的官方内置和全面支持。

Keras 有两个重要的概念： 模型（Model） 和 层（Layer） 。层将各种计算流程和变量进行了封装（例如基本的全连接层，CNN 的卷积层、池化层等），而模型则将各种层进行组织和连接，并封装成一个整体，描述了如何将输入数据通过各种层以及运算而得到输出。在需要模型调用的时候，使用 y_pred = model(X) 的形式即可。Keras 在 tf.keras.layers 下内置了深度学习中大量常用的的预定义层，同时也允许我们自定义层。

```python
class MyModel(tf.keras.Model):
    def __init__(self):
        super().__init__()     # Python 2 下使用 super(MyModel, self).__init__()
        # 此处添加初始化代码（包含 call 方法中会用到的层），例如
        # layer1 = tf.keras.layers.BuiltInLayer(...)
        # layer2 = MyCustomLayer(...)

    def call(self, input):
        # 此处添加模型调用的代码（处理输入并返回输出），例如
        # x = layer1(input)
        # output = layer2(x)
        return output

    # 还可以添加自定义的方法
```

model.variables 可以获取模型参数

## MLP 多层感知器

In [12]:
class MNISTLoader():
    def __init__(self):
        mnist = tf.keras.datasets.mnist
        (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()
        # MNIST中的图像默认为uint8（0-255的数字）。以下代码将其归一化到0-1之间的浮点数，并在最后增加一维作为颜色通道
        self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)      # [60000, 28, 28, 1]
        self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis=-1)        # [10000, 28, 28, 1]
        self.train_label = self.train_label.astype(np.int32)    # [60000]
        self.test_label = self.test_label.astype(np.int32)      # [10000]
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]

    def get_batch(self, batch_size):
        # 从数据集中随机取出batch_size个元素并返回
        index = np.random.randint(0, self.num_train_data, batch_size)
        return self.train_data[index, :], self.train_label[index]

In [13]:
np.random.randint(0, 1000, 20)

array([ 52, 296, 545,  72, 637, 397, 784, 445, 887, 523,  64, 482, 658,
       395, 288, 195, 125, 439, 131, 265])

In [14]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()    # Flatten层将除第一维（batch_size）以外的维度展平
        self.dense1 = tf.keras.layers.Dense(units=100, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)

    def call(self, inputs):         # [batch_size, 28, 28, 1]
        x = self.flatten(inputs)    # [batch_size, 784]
        x = self.dense1(x)          # [batch_size, 100]
        x = self.dense2(x)          # [batch_size, 10]
        output = tf.nn.softmax(x)
        return output

In [15]:
num_steps = 5
batch_size = 20
learning_rate = 0.001

model = MLP()
data_loader = MNISTLoader()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

num_batches = int(data_loader.num_train_data // batch_size * num_steps)
for batch_index in range(num_batches):
    X, y = data_loader.get_batch(batch_size)
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
        loss = tf.reduce_mean(loss)
        print("batch %d: loss %f" % (batch_index, loss.numpy()))
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))

batch 0: loss 2.501384
batch 1: loss 2.407601
batch 2: loss 2.263526
batch 3: loss 2.313154
batch 4: loss 2.220874
batch 5: loss 2.246051
batch 6: loss 2.092300
batch 7: loss 2.045517
batch 8: loss 2.156701
batch 9: loss 2.029128
batch 10: loss 2.018312
batch 11: loss 1.932396
batch 12: loss 1.904891
batch 13: loss 1.921469
batch 14: loss 1.890492
batch 15: loss 1.730428
batch 16: loss 1.876410
batch 17: loss 1.804725
batch 18: loss 1.523192
batch 19: loss 1.649372
batch 20: loss 1.475030
batch 21: loss 1.527068
batch 22: loss 1.491030
batch 23: loss 1.316585
batch 24: loss 1.415239
batch 25: loss 1.274964
batch 26: loss 1.335868
batch 27: loss 1.386894
batch 28: loss 1.286332
batch 29: loss 1.139123
batch 30: loss 1.442060
batch 31: loss 1.024244
batch 32: loss 1.142836
batch 33: loss 1.241083
batch 34: loss 1.087744
batch 35: loss 0.953354
batch 36: loss 0.916921
batch 37: loss 0.778706
batch 38: loss 1.099277
batch 39: loss 0.762406
batch 40: loss 0.761231
batch 41: loss 0.708605
ba

batch 334: loss 0.605910
batch 335: loss 0.668583
batch 336: loss 0.351332
batch 337: loss 0.090997
batch 338: loss 0.628002
batch 339: loss 0.492092
batch 340: loss 0.362881
batch 341: loss 0.163417
batch 342: loss 0.237124
batch 343: loss 0.210601
batch 344: loss 0.144943
batch 345: loss 0.332662
batch 346: loss 0.589515
batch 347: loss 0.238791
batch 348: loss 0.123025
batch 349: loss 0.201421
batch 350: loss 0.288574
batch 351: loss 0.127736
batch 352: loss 0.428766
batch 353: loss 0.354934
batch 354: loss 0.153769
batch 355: loss 0.473444
batch 356: loss 0.407014
batch 357: loss 0.427100
batch 358: loss 0.332219
batch 359: loss 0.376263
batch 360: loss 0.421803
batch 361: loss 0.130796
batch 362: loss 0.130561
batch 363: loss 0.336336
batch 364: loss 0.285279
batch 365: loss 0.197733
batch 366: loss 0.074347
batch 367: loss 0.464133
batch 368: loss 0.410874
batch 369: loss 0.367365
batch 370: loss 0.090998
batch 371: loss 0.412590
batch 372: loss 0.273957
batch 373: loss 0.487812


batch 690: loss 0.252486
batch 691: loss 0.176027
batch 692: loss 0.231251
batch 693: loss 0.168947
batch 694: loss 0.040816
batch 695: loss 0.161595
batch 696: loss 0.230100
batch 697: loss 0.735228
batch 698: loss 0.295736
batch 699: loss 0.408559
batch 700: loss 0.575828
batch 701: loss 0.040084
batch 702: loss 0.182547
batch 703: loss 0.374487
batch 704: loss 0.106897
batch 705: loss 0.174850
batch 706: loss 0.458516
batch 707: loss 0.599192
batch 708: loss 0.080719
batch 709: loss 0.049764
batch 710: loss 0.206146
batch 711: loss 0.256848
batch 712: loss 0.322371
batch 713: loss 0.051653
batch 714: loss 0.162899
batch 715: loss 0.250565
batch 716: loss 0.235927
batch 717: loss 0.665291
batch 718: loss 0.499380
batch 719: loss 0.218719
batch 720: loss 0.206752
batch 721: loss 0.157992
batch 722: loss 0.544110
batch 723: loss 0.594948
batch 724: loss 0.225089
batch 725: loss 0.405449
batch 726: loss 0.361446
batch 727: loss 0.572654
batch 728: loss 0.062345
batch 729: loss 0.040171


batch 1055: loss 0.176451
batch 1056: loss 0.145899
batch 1057: loss 0.302173
batch 1058: loss 0.586959
batch 1059: loss 0.129540
batch 1060: loss 0.160587
batch 1061: loss 0.154228
batch 1062: loss 0.461039
batch 1063: loss 0.098837
batch 1064: loss 0.319209
batch 1065: loss 0.523701
batch 1066: loss 0.190767
batch 1067: loss 0.113685
batch 1068: loss 0.235819
batch 1069: loss 0.072476
batch 1070: loss 0.065370
batch 1071: loss 0.055212
batch 1072: loss 0.146387
batch 1073: loss 0.555088
batch 1074: loss 0.224658
batch 1075: loss 0.192176
batch 1076: loss 0.418853
batch 1077: loss 0.267226
batch 1078: loss 0.127594
batch 1079: loss 0.105475
batch 1080: loss 0.121711
batch 1081: loss 0.378259
batch 1082: loss 0.526899
batch 1083: loss 0.192839
batch 1084: loss 0.362219
batch 1085: loss 0.206892
batch 1086: loss 0.177084
batch 1087: loss 0.415188
batch 1088: loss 0.411805
batch 1089: loss 0.221017
batch 1090: loss 0.251101
batch 1091: loss 0.045727
batch 1092: loss 0.305046
batch 1093: 

batch 1383: loss 0.069682
batch 1384: loss 0.075366
batch 1385: loss 0.182860
batch 1386: loss 0.175183
batch 1387: loss 0.267358
batch 1388: loss 0.041088
batch 1389: loss 0.070206
batch 1390: loss 0.076412
batch 1391: loss 0.314545
batch 1392: loss 0.160000
batch 1393: loss 0.155080
batch 1394: loss 0.179995
batch 1395: loss 0.258242
batch 1396: loss 0.081603
batch 1397: loss 0.089527
batch 1398: loss 0.130807
batch 1399: loss 0.036685
batch 1400: loss 0.084663
batch 1401: loss 0.053585
batch 1402: loss 0.227941
batch 1403: loss 0.059544
batch 1404: loss 0.207643
batch 1405: loss 0.176484
batch 1406: loss 0.047801
batch 1407: loss 0.418016
batch 1408: loss 0.094550
batch 1409: loss 0.167930
batch 1410: loss 0.184329
batch 1411: loss 0.123924
batch 1412: loss 0.457059
batch 1413: loss 0.025254
batch 1414: loss 0.393382
batch 1415: loss 0.170887
batch 1416: loss 0.116601
batch 1417: loss 0.337047
batch 1418: loss 0.131304
batch 1419: loss 0.132770
batch 1420: loss 0.207902
batch 1421: 

...

batch 13696: loss 0.020489
batch 13697: loss 0.210362
batch 13698: loss 0.064187
batch 13699: loss 0.004239
batch 13700: loss 0.009581
batch 13701: loss 0.029056
batch 13702: loss 0.022910
batch 13703: loss 0.007468
batch 13704: loss 0.006537
batch 13705: loss 0.152015
batch 13706: loss 0.000985
batch 13707: loss 0.145153
batch 13708: loss 0.002086
batch 13709: loss 0.017244
batch 13710: loss 0.003912
batch 13711: loss 0.020809
batch 13712: loss 0.108116
batch 13713: loss 0.005162
batch 13714: loss 0.015033
batch 13715: loss 0.045415
batch 13716: loss 0.048581
batch 13717: loss 0.011340
batch 13718: loss 0.061110
batch 13719: loss 0.141362
batch 13720: loss 0.072071
batch 13721: loss 0.006625
batch 13722: loss 0.006647
batch 13723: loss 0.036064
batch 13724: loss 0.008200
batch 13725: loss 0.063949
batch 13726: loss 0.021875
batch 13727: loss 0.034104
batch 13728: loss 0.009300
batch 13729: loss 0.159257
batch 13730: loss 0.000943
batch 13731: loss 0.107229
batch 13732: loss 0.001447
b

batch 14024: loss 0.096658
batch 14025: loss 0.009062
batch 14026: loss 0.082487
batch 14027: loss 0.070790
batch 14028: loss 0.006304
batch 14029: loss 0.187318
batch 14030: loss 0.018619
batch 14031: loss 0.001139
batch 14032: loss 0.006990
batch 14033: loss 0.091077
batch 14034: loss 0.252211
batch 14035: loss 0.010768
batch 14036: loss 0.026133
batch 14037: loss 0.011053
batch 14038: loss 0.008789
batch 14039: loss 0.017004
batch 14040: loss 0.154654
batch 14041: loss 0.019971
batch 14042: loss 0.017933
batch 14043: loss 0.003140
batch 14044: loss 0.002947
batch 14045: loss 0.012026
batch 14046: loss 0.022893
batch 14047: loss 0.009188
batch 14048: loss 0.015054
batch 14049: loss 0.005265
batch 14050: loss 0.016111
batch 14051: loss 0.012560
batch 14052: loss 0.025747
batch 14053: loss 0.012895
batch 14054: loss 0.021400
batch 14055: loss 0.243088
batch 14056: loss 0.051383
batch 14057: loss 0.008422
batch 14058: loss 0.001361
batch 14059: loss 0.003455
batch 14060: loss 0.010660
b

batch 14350: loss 0.000421
batch 14351: loss 0.103108
batch 14352: loss 0.045277
batch 14353: loss 0.029289
batch 14354: loss 0.044540
batch 14355: loss 0.001999
batch 14356: loss 0.027293
batch 14357: loss 0.002437
batch 14358: loss 0.006489
batch 14359: loss 0.004510
batch 14360: loss 0.015348
batch 14361: loss 0.048771
batch 14362: loss 0.005715
batch 14363: loss 0.000870
batch 14364: loss 0.004577
batch 14365: loss 0.021801
batch 14366: loss 0.001453
batch 14367: loss 0.042344
batch 14368: loss 0.025574
batch 14369: loss 0.008900
batch 14370: loss 0.008611
batch 14371: loss 0.014091
batch 14372: loss 0.036244
batch 14373: loss 0.017884
batch 14374: loss 0.070039
batch 14375: loss 0.002248
batch 14376: loss 0.008773
batch 14377: loss 0.094006
batch 14378: loss 0.040797
batch 14379: loss 0.017694
batch 14380: loss 0.058855
batch 14381: loss 0.000833
batch 14382: loss 0.133817
batch 14383: loss 0.039081
batch 14384: loss 0.167919
batch 14385: loss 0.007546
batch 14386: loss 0.031831
b

batch 14661: loss 0.001594
batch 14662: loss 0.012943
batch 14663: loss 0.007909
batch 14664: loss 0.014880
batch 14665: loss 0.046743
batch 14666: loss 0.006532
batch 14667: loss 0.006511
batch 14668: loss 0.038595
batch 14669: loss 0.050558
batch 14670: loss 0.010324
batch 14671: loss 0.034162
batch 14672: loss 0.030852
batch 14673: loss 0.008850
batch 14674: loss 0.011277
batch 14675: loss 0.096810
batch 14676: loss 0.003137
batch 14677: loss 0.012658
batch 14678: loss 0.004854
batch 14679: loss 0.021547
batch 14680: loss 0.042553
batch 14681: loss 0.025232
batch 14682: loss 0.134234
batch 14683: loss 0.001371
batch 14684: loss 0.003292
batch 14685: loss 0.026604
batch 14686: loss 0.005402
batch 14687: loss 0.044644
batch 14688: loss 0.028755
batch 14689: loss 0.000909
batch 14690: loss 0.214659
batch 14691: loss 0.010795
batch 14692: loss 0.003513
batch 14693: loss 0.003197
batch 14694: loss 0.124616
batch 14695: loss 0.031118
batch 14696: loss 0.025368
batch 14697: loss 0.003008
b

batch 14982: loss 0.098766
batch 14983: loss 0.049246
batch 14984: loss 0.176807
batch 14985: loss 0.003149
batch 14986: loss 0.001843
batch 14987: loss 0.034254
batch 14988: loss 0.006286
batch 14989: loss 0.047388
batch 14990: loss 0.035490
batch 14991: loss 0.010808
batch 14992: loss 0.001076
batch 14993: loss 0.101342
batch 14994: loss 0.018646
batch 14995: loss 0.077219
batch 14996: loss 0.000363
batch 14997: loss 0.000543
batch 14998: loss 0.026642
batch 14999: loss 0.016186


In [17]:
# 效果评估
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
num_batches = int(data_loader.num_test_data // batch_size)
for batch_index in range(num_batches):
    start_index, end_index = batch_index * batch_size, (batch_index + 1) * batch_size
    y_pred = model.predict(data_loader.test_data[start_index: end_index])
    sparse_categorical_accuracy.update_state(y_true=data_loader.test_label[start_index: end_index], y_pred=y_pred)
print("test accuracy: %f" % sparse_categorical_accuracy.result())

(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28, 28, 1)
(20, 28,

In [19]:
model.predict(data_loader.test_data[start_index: end_index])

array([[2.34854292e-09, 1.10537057e-09, 9.99004781e-01, 5.11432532e-04,
        3.67139019e-17, 1.34342440e-10, 9.40532713e-14, 4.83771961e-04,
        4.79876450e-09, 6.11794551e-14],
       [4.80050105e-04, 2.19292292e-06, 4.72321481e-05, 2.25487274e-05,
        5.58627335e-07, 7.18517636e-04, 9.98728693e-01, 8.09668599e-09,
        1.14703752e-07, 6.95960900e-10],
       [2.70263903e-04, 1.31596597e-02, 3.16118356e-04, 4.68412880e-03,
        7.67780966e-06, 3.46981525e-01, 6.33513033e-01, 1.05516688e-06,
        1.03255443e-03, 3.39846156e-05],
       [1.00000000e+00, 1.95573368e-14, 1.21338029e-09, 7.17794151e-14,
        2.27964426e-17, 1.74910889e-12, 1.17136179e-11, 5.71129422e-10,
        6.19514138e-14, 2.41529691e-12],
       [9.43677025e-10, 9.99446571e-01, 3.48649309e-07, 8.48385480e-06,
        1.45947581e-07, 2.10888977e-08, 3.63519576e-10, 5.26387303e-04,
        1.61024436e-05, 2.06322079e-06],
       [1.36609101e-11, 4.87370635e-05, 9.98971224e-01, 8.71684228e-04,
   

In [ ]:
tf.keras.applications.

In [ ]:
# CNN
class CNN(tf.keras.Model):
    def __init__(self):
        super.__init__()
        self.conv1 = 